In [9]:
import numpy as np
import aocd
import heapq

In [10]:
test_input = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""

In [ ]:
#import some stuff from day 16
def parse_input(data):
    return np.array([[x for x in line] for line in data.split("\n")])

def make_stringmap(arr):
    stringmap = []
    for line in arr:
        stringmap.append("".join(line))
    stringmap = "\n".join(stringmap)
    return stringmap

def turn_left(direction):
    # Define the 90-degree counterclockwise rotation matrix
    turn_left_matrix = np.array([[0, -1], [1, 0]])
    
    # Multiply the direction by the rotation matrix
    return tuple(np.dot(turn_left_matrix, direction))

def turn_right(direction):
    # Define the 90-degree clockwise rotation matrix
    turn_right_matrix = np.array([[0, 1], [-1, 0]])
    
    # Multiply the direction by the rotation matrix
    return tuple(np.dot(turn_right_matrix, direction))

#edit this like before to use only cost increase when moving forwards, keep track of path
def dijkstra(arr):
    start = np.argwhere(arr == "S")[0]
    visited = set()
    q = [(0, [start], [0,1])] #cost, pos, direction
    while q:
        cost, path, direction = heapq.heappop(q) #retrieve the lowest cost next step from the queue for checking
        loc = path[-1]
        if (tuple(loc), tuple(direction)) in visited: #if we have already been here, we are on the way back from a bad route and need to ignore the loc
            continue #we have done this
        if arr[loc[0], loc[1]] == "E": #if we stumble upon the target tile, we have found the (or at least one) lowest cost route.
            #arrived, we return cost
            return cost, path
        visited.add((tuple(loc), tuple(direction))) #we record our current position and direction
        one_ahead = [loc[0] + direction[0], loc[1]+direction[1]] #look at the space one ahead of us:
        if arr[one_ahead[0], one_ahead[1]] != "#":
            heapq.heappush(q, (cost+1, path+[[one_ahead[0], one_ahead[1]]], direction)) #if the space one ahead of us is not a wall, it is an empty space or the end, and we need to explore it for a cost of 1
        #we also need to explore what happens if we turn left and right, for a cost of 1000 each.
        heapq.heappush(q, (cost+1, path, turn_left(direction)))
        heapq.heappush(q, (cost+1, path, turn_right(direction)))

In [168]:
arr = parse_input(test_input)

In [169]:
cost, path = dijkstra(arr)

In [170]:
path = np.vstack(path)

In [171]:
def manhattan(p1, p2):
    return np.sum(np.abs(p1-p2), axis = 1)

In [172]:
hacks = set() 
hack_data = []
for i, p in enumerate(path):
    dists = manhattan(path, p)
    hack_available = np.ravel(np.argwhere(dists==2))
    for hack in hack_available:
        hack_dist = hack - i - 2
        if hack_dist > 0 and (i, int(hack)) not in hacks:
            hacks.add((hack, i)) #reverse this to catch future duplicates
            hack_data.append({"hack": hack, "path_val":p , "i": i, "saved": hack_dist})


In [173]:
def solve1(puzzle_input):
    arr = parse_input(puzzle_input)
    cost,path = dijkstra(arr)
    path = np.vstack(path)
    hacks = set() 
    hack_data = []
    for i, p in enumerate(path):
        dists = manhattan(path, p)
        hack_available = np.ravel(np.argwhere(dists==2))
        for hack in hack_available:
            hack_dist = hack - i - 2
            if hack_dist > 0 and (i, int(hack)) not in hacks:
                hacks.add((hack, i)) #reverse this to catch future duplicates
                hack_data.append({"hack": hack, "path_val":p , "i": i, "saved": hack_dist})
    return pd.DataFrame.from_dict(hack_data)

In [174]:
df = solve1(test_input)

In [175]:
df.groupby("saved").count()

,hack,path_val,i
saved,,,
2,14,14,14
4,14,14,14
6,2,2,2
8,4,4,4
10,2,2,2
12,3,3,3
20,1,1,1
36,1,1,1
38,1,1,1


In [176]:
puzzle_input = aocd.get_data()

In [177]:
df = solve1(puzzle_input)

In [178]:
df[df["saved"] >= 100]

,hack,path_val,i,saved
6,312,"[63, 115]",6,304
7,328,"[63, 115]",6,320
8,311,"[63, 116]",7,302
9,146,"[63, 117]",8,136
10,310,"[63, 117]",8,300
...,...,...,...,...
6731,9165,"[89, 124]",9057,106
6732,9164,"[89, 125]",9058,104
6733,9163,"[89, 126]",9059,102
6735,9162,"[89, 127]",9060,100


In [186]:
def solve2(puzzle_input, max_hack_length): #adaot to generalize for hack length
    arr = parse_input(puzzle_input)
    cost,path = dijkstra(arr)
    path = np.vstack(path)
    hacks = set() 
    hack_data = []
    for i, p in enumerate(path):
        dists = manhattan(path, p)
        hack_available = np.ravel(np.argwhere(dists<= max_hack_length)) #check hacks up to hack length
        for hack in hack_available:
            hack_dist = hack - i - dists[hack] #calculate the saved distance, accounting for the hack length
            if (i, int(hack)) not in hacks:
                hacks.add((hack, i)) #reverse this to catch future duplicates
                hack_data.append({"hack": hack, "path_val":p , "i": i, "saved": hack_dist})
    return pd.DataFrame.from_dict(hack_data)

In [187]:
df = solve2(test_input, max_hack_length=20)

In [188]:
df[df.saved >= 50].groupby("saved").count()

,hack,path_val,i
saved,,,
50,32,32,32
52,31,31,31
54,29,29,29
56,39,39,39
58,25,25,25
60,23,23,23
62,20,20,20
64,19,19,19
66,12,12,12


In [189]:
df = solve2(puzzle_input, max_hack_length=20)

df[df["saved"] >= 100].groupby("saved").count()["hack"].sum()

np.int64(944910)

In [191]:
len(df[df["saved"] >= 100])

944910